In [1]:
from torch import optim
import torch.utils.data as data
from setting_model import *
import numpy as np
import os

CUDA Device Count: 1
CUDA Device Name: NVIDIA GeForce RTX 3060 Laptop GPU


In [2]:
# Model instantiation
# forward_model = model
forward_model = FC_Net().to(device)

iteration = 0
interval = 100
batch_size = 20                 # 修改
lr = 3e-4
epoch = 2000
iter_num = []
losses = {'err_train': [], 'err_test': [], 'loss_train': [], 'loss_test': []}

In [3]:
# Gets the relevant file path
data_root = os.getcwd()
bo_len_path = os.path.join(data_root, "Train_Data_Fig4", "X.csv")
assert os.path.exists(bo_len_path), "{} path does not exist.".format(bo_len_path)                  # assert function

train_spectrum_path = os.path.join(data_root, "Train_Data_Fig4", "train_680.csv")
assert os.path.exists(train_spectrum_path), "{} path does not exist.".format(train_spectrum_path)

test_spectrum_path = os.path.join(data_root, "Train_Data_Fig4", "test_77.csv")
assert os.path.exists(test_spectrum_path), "{} path does not exist.".format(test_spectrum_path)

# Save the updated weights file
#save_dir = os.path.join(data_root, "weight_optim")
#assert os.path.exists(save_dir), "{} path does not exist.".format(save_dir)

In [4]:
# Loader Data
train_numpy = np.loadtxt(train_spectrum_path, delimiter=",")
train_input = torch.FloatTensor(train_numpy[:, 600:603]).to(device)
train_label = torch.FloatTensor(train_numpy[:, 300:600]).to(device)
train_dataset = data.TensorDataset(train_input, train_label)
data_loader = data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)    # 提取训练数据集并加载

test_numpy = np.loadtxt(test_spectrum_path, delimiter=",")
test_input = torch.FloatTensor(test_numpy[:, 600:603]).to(device)
test_label = torch.FloatTensor(test_numpy[:, 300:600]).to(device)                               # 提取测试数据集

In [5]:
train_input.shape, test_input.shape

(torch.Size([680, 3]), torch.Size([77, 3]))

In [6]:
# Training process
def train():
    optimizer = optim.Adam(forward_model.parameters(), lr=lr)
    criterion = nn.MSELoss(reduction='mean')

    for t in range(epoch):
        for step, item in enumerate(data_loader):
            # train
            train_input, train_label = item
            train_predict = forward_model(train_input)
            loss_train = criterion(train_predict, train_label)
            
            global iteration                      # 声明全局变量， 不声明全局变量会报错
            if iteration % interval == 0:
                # test
                test_predict = forward_model(test_input)
                loss_test = criterion(test_predict, test_label)
                losses['loss_train'].append(loss_train.cpu().detach().numpy())
                losses['loss_test'].append(loss_test.cpu().detach().numpy())

                # compute and print the absolute error
                train_out = train_predict - train_label
                train_error = np.abs(train_out.cpu().detach().numpy()).mean()
                test_out = test_predict - test_label
                test_error = np.abs(test_out.cpu().detach().numpy()).mean()
                losses['err_train'].append(train_error)
                losses['err_test'].append(test_error)

                print('iteration: {}'.format(iteration))
                print('train_loss: {:.4}, test_loss: {:.4}'.
                      format(loss_train, loss_test))
                print('train_error: {:.4}, test_error: {:.4}'.
                      format(train_error, test_error))

                iter_num.append(iteration)


            # update parameters
            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()

            iteration += 1

In [7]:
train()

iteration: 0
train_loss: 0.2346, test_loss: 0.1717
train_error: 0.2774, test_error: 0.2193
iteration: 100
train_loss: 0.03766, test_loss: 0.03489
train_error: 0.1347, test_error: 0.1085
iteration: 200
train_loss: 0.01622, test_loss: 0.01734
train_error: 0.08465, test_error: 0.08711
iteration: 300
train_loss: 0.01525, test_loss: 0.01655
train_error: 0.06669, test_error: 0.08055
iteration: 400
train_loss: 0.01252, test_loss: 0.0131
train_error: 0.07214, test_error: 0.07148
iteration: 500
train_loss: 0.005183, test_loss: 0.006682
train_error: 0.04815, test_error: 0.05473
iteration: 600
train_loss: 0.006231, test_loss: 0.008633
train_error: 0.05506, test_error: 0.05742
iteration: 700
train_loss: 0.002313, test_loss: 0.004276
train_error: 0.03318, test_error: 0.04426
iteration: 800
train_loss: 0.003474, test_loss: 0.003067
train_error: 0.0409, test_error: 0.03755
iteration: 900
train_loss: 0.005753, test_loss: 0.004188
train_error: 0.05004, test_error: 0.04261
iteration: 1000
train_loss: 0.

KeyboardInterrupt: 

In [ ]:
# 过拟合
# MSEloss curve
Len = len(losses['loss_train'])
epoch_total = np.linspace(0, Len, Len)
plt.title('MSEloss')
plt.plot(epoch_total, losses['loss_train'],color="black", label='Train_loss')
plt.plot(epoch_total, losses['loss_test'],color="orange", label='Train_loss')
plt.legend(loc='upper right')
plt.xlabel('epoch_total')
plt.ylabel('loss_fn')
plt.show()

In [ ]:
# train or test
bo_len = np.loadtxt(bo_len_path, delimiter=",", dtype=np.float32)

# plot_train_data
c = 3
valid_input = train_input[c:c+1]
valid_predict = forward_model(valid_input).view(-1).cpu().detach().numpy().reshape(1, 300)               
valid_label = train_label[c].cpu().detach().numpy().reshape(1, 300)                                    
valid_predict = valid_predict.reshape(300,)
valid_label = valid_label.reshape(300,)
print(valid_input)

plt.title('Comparison of Transmission Spectrum')
plt.plot(bo_len, valid_predict, color="orange", label='Prediction', linestyle='-')
plt.plot(bo_len, valid_label, color="black", label='Simulation', linestyle=':')
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('Fx(10^-12)')
plt.show()

In [ ]:
test_input.shape

In [ ]:
# 测试及画图
c = 36                              # 36
valid_input = test_input[c:c+1]
print(valid_input)
valid_predict = forward_model(valid_input).view(-1).cpu().detach().numpy().reshape(1, 300)               
valid_label = test_label[c].cpu().detach().numpy().reshape(1, 300)                                    
valid_predict = valid_predict.reshape(300,)
valid_label = valid_label.reshape(300,)

plt.title('Comparison of Transmission Spectrum')
plt.plot(bo_len, valid_predict, color="orange", label='Prediction', linestyle='--')
plt.plot(bo_len, valid_label, color="black", label='Simulation', linestyle=':')
plt.legend(loc='upper right')
plt.xlabel('x')
plt.ylabel('Fx(10^-12)')
plt.show()

In [ ]:
# Save the updated weights file
save_dir = os.path.join(data_root, "weight_optim")
assert os.path.exists(save_dir), "{} path does not exist.".format(save_dir)
# Save the weight_optim
torch.save(forward_model, save_dir + '\model_train_Fy'  + '.pth')